<a id='topo'></a>

# [MobileNet - (128x128)](#sem_imagenet)
- rede alterada para grayscale (Input com 1 channel)
- realizado resize das imagens para 128x128
- fit com 10 epochs


# [MobileNet - (128x128) Pesos imagenet](#pesos_imagenet)
- rede alterada para grayscale (Input com 1 channel)
- realizado resize das imagens para 128x128
- fit com 10 epochs
- adicionados pesos imagenet e camadas finais (include_top = True)

In [ ]:
# Configura conjuntos de treinamento e teste 
# X_train, X_test, Y_train, Y_test
%run './functions/SLD-LoadTrainTestSets.ipynb'

In [ ]:
import keras
from keras import backend as K
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Model, Input
from keras.applications.mobilenet import MobileNet
from keras.layers import Conv2D, GlobalAveragePooling2D, Reshape
from keras.optimizers import Adam

In [ ]:
X_train = np.array(resize_npy_images(X_train, (128, 128)))
X_test = np.array(resize_npy_images(X_test, (128, 128)))

In [ ]:
print('Training shape:', X_train.shape)
print('Test shape:', X_test.shape)

In [ ]:
original_input_shape = (128, 128, 3)
desired_input_shape = (128, 128, 1)
layer_name = 'conv1'

weights = None
include_top = False

adam_lr = 0.0001
nb_epochs = 10
cm_plot_labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
validation_split = 0.1
batch_size=10

---

<a id='sem_imagenet'></a>

# Rede sem pesos ImageNet e sem Include top

[Voltar para topo](#topo)

In [ ]:
base_model = MobileNet(input_shape=original_input_shape,
                       weights=weights,
                       include_top=include_top)
base_model.layers[-1].name

In [ ]:
# Alterando input para grayscale (1 channel)
selected_layer = base_model.get_layer(layer_name)
selected_layer_index = base_model.layers.index(selected_layer)

w = selected_layer.get_weights()
w[0] = w[0].mean(axis=2, keepdims=True)

y = x = Input(shape=desired_input_shape)

attributes = dict(selected_layer.get_config(),
                  name=layer_name + '_1',
                  weights=w)
selected_layer = Conv2D.from_config(attributes)

y = selected_layer(y)

for layer in base_model.layers[selected_layer_index + 1:]:
    y = layer(y)

In [ ]:
# Adicionando camadas representadas pelo include_top=True
# camada FC no final da rede
y = GlobalAveragePooling2D()(y)
y = Reshape((1, 1, 1024))(y)
y = Dropout(0.25)(y)
y = Conv2D(10, kernel_size=(1, 1), activation='relu')(y)
y = Activation('softmax')(y)
y = Reshape((10,))(y)

In [ ]:
model = Model(inputs=x, outputs=y)
model.summary()

In [ ]:
# Congela camadas
# for layer in model.layers:
#     layer.trainable = False

In [ ]:
expand_train_X = np.expand_dims(X_train, axis=-1)
expand_test_X = np.expand_dims(X_test, axis=-1)

In [ ]:
%%time
acertos, pred = execute_model(model, expand_train_X, Y_train, expand_test_X, Y_test,
                              validation_split, batch_size, cm_plot_labels, 
                              adam_lr, nb_epochs)

---

<a id='pesos_imagenet'></a>

# Rede com pesos ImageNet e com Include top

[Voltar para topo](#topo)

In [ ]:
weights = 'imagenet'
include_top = True

In [ ]:
base_model = MobileNet(input_shape=original_input_shape,
                       weights=weights,
                       include_top=include_top)
base_model.layers[-1].name

In [ ]:
base_model.summary()

In [ ]:
# Alterando input para grayscale (1 channel)
selected_layer = base_model.get_layer(layer_name)
selected_layer_index = base_model.layers.index(selected_layer)

w = selected_layer.get_weights()
w[0] = w[0].mean(axis=2, keepdims=True)

y = x = Input(shape=desired_input_shape)

attributes = dict(selected_layer.get_config(),
                  name=layer_name + '_1',
                  weights=w)
selected_layer = Conv2D.from_config(attributes)

y = selected_layer(y)

for layer in base_model.layers[selected_layer_index + 1:-4]:
    y = layer(y)

In [ ]:
# Adicionando camadas representadas pelo include_top=True
# camada FC no final da rede
# y = GlobalAveragePooling2D()(y)
# y = Reshape((1, 1, 1024))(y)
y = Dropout(0.25)(y)
y = Conv2D(10, kernel_size=(1, 1), activation='relu')(y)
y = Activation('softmax')(y)
y = Reshape((10,))(y)

In [ ]:
model = Model(inputs=x, outputs=y)
model.summary()

In [ ]:
# Congela camadas
# for layer in model.layers:
#     layer.trainable = False

In [ ]:
expand_train_X = np.expand_dims(X_train, axis=-1)
expand_test_X = np.expand_dims(X_test, axis=-1)

In [ ]:
%%time
acertos, pred = execute_model(model, expand_train_X, Y_train, expand_test_X, Y_test,
                              validation_split, batch_size, cm_plot_labels, 
                              adam_lr, nb_epochs)

---

In [ ]:
307/310

[Voltar para topo](#topo)

# Cria arquivo h5 com modelo da CNN

In [ ]:
model.save('SLD_MOBILENET_IMAGENET_CNN.h5')